In [ ]:
import geopandas as gpd
import polars as pl
import polars_st as st
import pyogrio

In [ ]:
# ASGA_FILE_PATH = "data/raw/ASGA/{year}/{region}/"
SA1_FILE_PATH = "/home/andre/git/private/electoralyze/data/raw/ASGA/2021/SA1/SA1_2021_AUST_GDA2020.shp"
VIC_ELECTION = "/home/andre/git/private/electoralyze/data/raw/AEC_boundary/federal/2024/VIC/E_VIC24_region.shp"

In [ ]:
sa1 = pyogrio.read_dataframe(SA1_FILE_PATH)
sa1

In [ ]:
sa1_st = st.GeoDataFrame(
    sa1[["SA1_CODE21", "geometry"]]
    .dropna()
    .to_crs(4326)
    .assign(SA1_CODE21=lambda df: df["SA1_CODE21"].astype(float))
    .to_dict("records")
)
sa1_st

In [ ]:
sa1_st_4326 = sa1_st.with_columns(st.geom("geometry").st.set_srid(4326))

In [ ]:
sa1_gdf: gpd.GeoDataFrame = sa1_st_4326.st.to_geopandas().set_crs(4326)
sa1_gdf.to_file("temp/100/sa1_processed.shp")
sa1_gdf.simplify(0.0001).to_file("temp/0_0001/sa1_processed.shp")
sa1_gdf.simplify(0.0005).to_file("temp/0_0005/sa1_processed.shp")

In [ ]:
sa1_gdf.simplify(0.0001).explore()

In [ ]:
import os


def get_folder_size(folder_path):
    total_size = 0
    for dirpath, _dirnames, filenames in os.walk(folder_path):
        for file in filenames:
            file_path = os.path.join(dirpath, file)
            # Check if it's a file before getting its size
            if os.path.isfile(file_path):
                total_size += os.path.getsize(file_path)
    return total_size

In [15]:
print(f"Folder size: {get_folder_size("temp/100") / (1024 ** 2):.2f} MB")  # Convert to MB
print(f"Folder size: {get_folder_size("temp/0_0001") / (1024 ** 2):.2f} MB")  # Convert to MB
print(f"Folder size: {get_folder_size("temp/0_0005") / (1024 ** 2):.2f} MB")  # Convert to MB

Folder size: 152.02 MB
Folder size: 45.16 MB
Folder size: 24.92 MB
